# SAT Solvers - Exercício de avaliação

Número: pg52669

Nome: Afonso Nogueira Ferreira

Curso: Mestrado em Engenharia Informática

In [ ]:
!pip install z3-solver

#Coloração de Azulejos

1. Cada azulejo tem uma única cor.
2. Os azulejos das extremidades têm que ter a mesma cor.
3. Todas as cores têm que ser usadas.
4. O azulejo do meio só pode ser pintado de cinzento ou preto.
5. O penúltimo azulejo da sequência tem que ser preto, excepto se o último estiver pintado
de preto.
6. Azulejos seguidos não podem ser pintados da mesma cor.
7. Caso o primeiro azulejo seja verde e o terceiro não seja preto, o segundo terá que ser
cinzento ou preto.
8. Os azulejos na terceira e na sétima posição só podem ser pintados de azul ou de verde.

In [ ]:
from z3 import *

azulejos = [1,2,3,4,5,6,7,8,9]
cores = ["Azul", "Verde", "Cinzento", "Preto"]
x = {}
for a in azulejos:
  x[a] = {}
  for c in cores:
    x[a][c] = Bool("%d,%s" % (a,c))

s = Solver()

# Cada azulejo tem pelo menos uma cor.
for a in azulejos:
  s.add(Or(x[a]["Azul"],x[a]["Verde"],x[a]["Cinzento"],x[a]["Preto"]))

# Cada azulejo tem uma única cor.
for a in azulejos:
    for i in range(len(cores)-1):
        for j in range(i+1,len(cores)):
            s.add(Implies(x[a][cores[i]],Not(x[a][cores[j]])))

# Os azulejos das extremidades têm que ter a mesma cor.
for c in cores:
  s.add(Implies(x[1][c],x[9][c]))

# Todas as cores têm que ser usadas.
for c in cores:
   s.add(Or([x[a][c] for a in azulejos]))

# O azulejo do meio só pode ser pintado de cinzento ou preto.
s.add(Or(x[5]["Cinzento"],x[5]["Preto"]))

# O penúltimo azulejo da sequência tem que ser preto, excepto se o último estiver pintado de preto.
s.add(Implies(Not(x[9]["Preto"]),x[8]["Preto"]))

# Azulejos seguidos não podem ser pintados da mesma cor.
for c in cores:
  for i in range(len(azulejos)-1):
    s.add(Implies(x[azulejos[i]][c],Not(x[azulejos[i+1]][c])))

# Caso o primeiro azulejo seja verde e o terceiro não seja preto, o segundo terá que ser cinzento ou preto.
s.add(Implies(And(x[1]["Verde"],Not(x[3]["Preto"])),Or(x[2]["Cinzento"],x[2]["Preto"])))

# Os azulejos na terceira e na sétima posição só podem ser pintados de azul ou de verde.
s.add(And(Or(x[3]["Azul"],x[3]["Verde"]),Or(x[7]["Azul"],x[7]["Verde"])))


1. Codifique o problema num solver e comprove que o conjunto de fórmulas é consistente

In [ ]:
if s.check() == sat:
  m = s.model()
  print("O conjunto de fórmulas é consistente.")
  print("Exemplo:")
  for a in azulejos:
    for c in cores:
      if is_true(m[x[a][c]]):
        print("O azulejo %d foi pintado de %s" % (a,c))


O conjunto de fórmulas é consistente.
Exemplo:
O azulejo 1 foi pintado de Azul
O azulejo 2 foi pintado de Cinzento
O azulejo 3 foi pintado de Verde
O azulejo 4 foi pintado de Azul
O azulejo 5 foi pintado de Cinzento
O azulejo 6 foi pintado de Verde
O azulejo 7 foi pintado de Azul
O azulejo 8 foi pintado de Preto
O azulejo 9 foi pintado de Azul


2. Com a ajuda do solver, pronuncie-se quanto à veracidade das seguintes afirmações:

> a) Se o primeiro azulejo for preto, o penúltimo azulejo tem que ser azul.

In [ ]:
s.push()

form1 = Implies(x[1]["Preto"],x[8]["Azul"])

s.add(Not(form1))

if s.check() == unsat:
  print("A afirmação é verdadeira")
else:
  print("A afirmação é falsa")

s.pop()

A afirmação é falsa


> b) Algum dos dois últimos azulejos tem que ser preto.

In [ ]:
s.push()

form2 = Or(x[8]["Preto"],x[9]["Preto"])

s.add(Not(form2))

if s.check() == unsat:
  print("A afirmação é verdadeira")
else:
  print("A afirmação é falsa")

s.pop()

A afirmação é verdadeira


> c) Algum dos três primeiros azulejos poderá ser azul.

In [ ]:
s.push()

form3 = Or(Or([x[a]["Azul"] for a in azulejos[:3]]),And([Not(x[a]["Azul"]) for a in azulejos[:3]]))

s.add(Not(form3))

if s.check() == unsat:
  print("A afirmação é verdadeira")
else:
  print("A afirmação é falsa")

s.pop()

A afirmação é verdadeira


3. Acrescente código ao programa de modo a saber todas as colorações que são possíveis fazer aos azulejos. Quantas são?

In [ ]:
s.push()
i=0

while s.check() == sat:
  i += 1
  m = s.model()
  constraints = []  # Clear the constraints list for each iteration
  for a in azulejos:
    for c in cores:
      if is_true(m[x[a][c]]):
        constraints.append(Not(x[a][c]))  # Append constraints to the list
      else:
        constraints.append(x[a][c])  # Append constraints to the list

  s.add(Or(constraints))  # Add the constraints list to the solver
else:
  print("Número de soluções:", i)

Número de soluções: 306
